In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
import datetime as dt
from matplotlib import pyplot as plt

In [2]:
train = pd.read_csv("train.csv")
train

,datetime,total
0,01.01.2005 00:00:00,?
1,01.01.2005 01:00:00,154140
2,01.01.2005 02:00:00,157818
3,01.01.2005 03:00:00,149311
4,01.01.2005 04:00:00,138282
...,...,...
35059,31.12.2008 19:00:00,249376
35060,31.12.2008 20:00:00,246511
35061,31.12.2008 21:00:00,226469
35062,31.12.2008 22:00:00,199907


In [3]:
#time_format = "%d.%m.%Y %H:%M:%S"
#train["datetime"] = [dt.datetime.strptime(s, time_format) for s in train["datetime"]]
train["datetime"] = pd.to_datetime(train["datetime"], format='%d.%m.%Y %H:%M:%S')

In [5]:
train['month'] = pd.DatetimeIndex(train["datetime"]).month.astype(int)
train["weekDay"] = pd.DatetimeIndex(train["datetime"]).weekday.astype(int)
train["day"] = pd.DatetimeIndex(train["datetime"]).day.astype(int)
train["year"] = pd.DatetimeIndex(train["datetime"]).year.astype(int)

train["hour"] = [date.hour for date in train["datetime"]]
train["hour"].astype(int)
train['total'] = train['total']

In [6]:
train.head(50)

,datetime,total,month,weekDay,day,year,hour
0,2005-01-01 00:00:00,?,1,5,1,2005,0
1,2005-01-01 01:00:00,154140,1,5,1,2005,1
2,2005-01-01 02:00:00,157818,1,5,1,2005,2
3,2005-01-01 03:00:00,149311,1,5,1,2005,3
4,2005-01-01 04:00:00,138282,1,5,1,2005,4
5,2005-01-01 05:00:00,132033,1,5,1,2005,5
6,2005-01-01 06:00:00,129193,1,5,1,2005,6
7,2005-01-01 07:00:00,131026,1,5,1,2005,7
8,2005-01-01 08:00:00,138040,1,5,1,2005,8
9,2005-01-01 09:00:00,158872,1,5,1,2005,9


In [6]:
#train.iloc(np.where(train["total"] == '?'))

In [ ]:
# replace unknown and negative or zero consumtion with mean of such days of year (1 january, for examle)

for i in np.where(train['total']=='?')[0]:
    t = np.mean(list(map(float, train.loc[\
                (train['day'] == train['day'][i]) \
               & (train['month'] == train['month'][i])\
               & (train['hour'] == train['hour'][i])\
               & (train['total']!='?')]['total'])))
    
    train['total'][i] = t
    print(train['total'][i])
    #train['total'][i] = np.mean()
    
train['total'] = train['total'].astype(float)

for i in np.where(train['total']<=0)[0]:
    t = np.mean(list(map(float, train.loc[\
                (train['day'] == train['day'][i]) \
               & (train['month'] == train['month'][i])\
               & (train['hour'] == train['hour'][i])\
               & (train['total']>=0)]['total'])))
    
    train['total'][i] = t
    print(train['total'][i])

In [8]:
train

,datetime,total,month,weekDay,day,year,hour
0,2005-01-01 00:00:00,132407.0,1,5,1,2005,0
1,2005-01-01 01:00:00,154140.0,1,5,1,2005,1
2,2005-01-01 02:00:00,157818.0,1,5,1,2005,2
3,2005-01-01 03:00:00,149311.0,1,5,1,2005,3
4,2005-01-01 04:00:00,138282.0,1,5,1,2005,4
...,...,...,...,...,...,...,...
35059,2008-12-31 19:00:00,249376.0,12,2,31,2008,19
35060,2008-12-31 20:00:00,246511.0,12,2,31,2008,20
35061,2008-12-31 21:00:00,226469.0,12,2,31,2008,21
35062,2008-12-31 22:00:00,199907.0,12,2,31,2008,22


In [9]:
train.pop('datetime')

0       2005-01-01 00:00:00
1       2005-01-01 01:00:00
2       2005-01-01 02:00:00
3       2005-01-01 03:00:00
4       2005-01-01 04:00:00
                ...        
35059   2008-12-31 19:00:00
35060   2008-12-31 20:00:00
35061   2008-12-31 21:00:00
35062   2008-12-31 22:00:00
35063   2008-12-31 23:00:00
Name: datetime, Length: 35064, dtype: datetime64[ns]

In [10]:
train.head(30)

,total,month,weekDay,day,year,hour
0,132407.0,1,5,1,2005,0
1,154140.0,1,5,1,2005,1
2,157818.0,1,5,1,2005,2
3,149311.0,1,5,1,2005,3
4,138282.0,1,5,1,2005,4
5,132033.0,1,5,1,2005,5
6,129193.0,1,5,1,2005,6
7,131026.0,1,5,1,2005,7
8,138040.0,1,5,1,2005,8
9,158872.0,1,5,1,2005,9


In [11]:
x_train, x_test = train[:25000], train[25000:]
y_train, y_test = x_train.total, x_test.total
x_test.pop('total')
x_train.pop('total')

0        132407.0
1        154140.0
2        157818.0
3        149311.0
4        138282.0
           ...   
24995    209298.0
24996    204649.0
24997    199715.0
24998    204503.0
24999    188659.0
Name: total, Length: 25000, dtype: float64

In [12]:
x_test.tail()

,month,weekDay,day,year,hour
35059,12,2,31,2008,19
35060,12,2,31,2008,20
35061,12,2,31,2008,21
35062,12,2,31,2008,22
35063,12,2,31,2008,23


In [13]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x_train, y_train)
pred_linear = model.predict(x_test)

In [14]:
from sklearn.tree import DecisionTreeRegressor
tree=DecisionTreeRegressor()
tree.fit(x_train,y_train)
pred_tree = tree.predict(x_test)

In [19]:
y_test = y_test.values

In [20]:
pred = pred_tree

score = 1000
error = 0
for i in range(len(y_test)):
    error += abs(y_test[i]-pred[i])/(y_test[i]+pred[i])
score = 1000*(1-error/len(y_test))
score

963.5869658252492

In [22]:
#31,28, 31, 30, 31, 30
WEEKDAY = 3
dat = []
for m in range(1,7):
    for day in range(1, 32):
        WEEKDAY = (WEEKDAY+1)%7
        if (m==4 or m==6) and day==31:
            break
        if m==2 and day==29:
            break
        for hour in range(0, 24):
            dat.append([m,WEEKDAY,day,2009,hour])
            
df = pd.DataFrame(data = dat, columns = ['month', 'weekDay','day', 'year','hour'])
df

,month,weekDay,day,year,hour
0,1,4,1,2009,0
1,1,4,1,2009,1
2,1,4,1,2009,2
3,1,4,1,2009,3
4,1,4,1,2009,4
...,...,...,...,...,...
4339,6,4,30,2009,19
4340,6,4,30,2009,20
4341,6,4,30,2009,21
4342,6,4,30,2009,22


In [23]:
final_pred = tree.predict(df)
final_pred

array([135295., 136421., 144580., ..., 190723., 189927., 178037.])

In [24]:
with open("answer.txt", "w") as f:
    for v in final_pred:
        f.write(str(v))
        f.write('\n')